In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as TF
import keras
import cv2
from typing import Tuple
import os
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

CATEGORIES = ["NORMAL", "PNEUMONIA"]
EPOCHS = 50
TEST_SET = 0.3

In [43]:
def LoadDataFromDirectory(dataDir : str):
    images = []
    labels = []
    for Category in CATEGORIES:
        singleCategory = os.path.join(dataDir, str(Category))
        
        for image in os.listdir(singleCategory):
            imgPaths = os.path.join(singleCategory, image)
            
            finalImage = cv2.imread(imgPaths)
            
            if finalImage is None:
                raise Exception("The directories are Null or Error")
            img = cv2.resize(finalImage, (64,64))
            images.append(img)
            
            labels.append(CATEGORIES.index(Category))
        
    return (np.array(images), np.array(labels))

In [44]:
images, labels = LoadDataFromDirectory("train")

X_train, X_test, Y_train, Y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SET, train_size=1-TEST_SET
)

print(f"X train shape: {X_train.shape}")
print(f"Y train shape: {Y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"Y test shape: {Y_test.shape}")


X train shape: (103, 64, 64, 3)
Y train shape: (103,)
X test shape: (45, 64, 64, 3)
Y test shape: (45,)


In [ ]:
def model(input_shape): 
    input_img = TF.keras.Input(shape=input_shape)
    
    # CONV: f=4, stride=1 (default), padding=same
    Z1 = TF.keras.layers.Conv2D(filters=10, kernel_size=(4,4), strides=(1,1), padding="same")(input_img)
    
    A1 = TF.keras.layers.ReLU()(Z1)
    
    # POOL: f=10, stride=10, padding=valid
    P1 = TF.keras.layers.MaxPool2D(pool_size=(10,10), strides=10, padding="valid")(A1)
    
    # CONV: f=2, stride=1 (default), padding=same
    Z2 = TF.keras.layers.Conv2D(filters=16, kernel_size=(2,2), strides=(1,1), padding="valid")(P1)
    
    A2 = TF.keras.layers.ReLU()(Z2)
    
    # POOL: f=5, stride=5, padding=valid
    P2 = TF.keras.layers.MaxPool2D(pool_size=(5,5), strides=(5,5), padding="same")(A2)
    
    F = TF.keras.layers.Flatten()(P2)

    output = TF.keras.layers.Dense(units = 1, activation="sigmoid")(F)
    
    model = TF.keras.Model(input_img, output)
    return model
    
    

In [42]:
ConvModel = model((64,64,3))

ConvModel.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
Y_train_oh = to_categorical(Y_train, num_classes=2)
Y_test_oh = to_categorical(Y_test, num_classes=2)

history = ConvModel.fit(X_train/255.0, Y_train, epochs=EPOCHS, validation_data=(X_test/255.0, Y_test))
ConvModel.summary()


ValueError: invalid literal for int() with base 10: 'train/NORMAL'